# Árvore de decisão

### Base de dados: State of Data Brazil 2023

Prof. Hugo de Paula

Prof. Hayala

Descrição dos atributos:
1-Idade{>=18 & <=73}

2-Genero{Masculino, Feminino}

3-Cor/raca/etnia{Branca,Preta,Parda,Prefiro não informar,Amarela,Indígena,Outra}

4-experiencia_profissional_prejudicada(0,1,2}


-------------------------------------------------------------------------------

### Inicialização da plataforma do Colab.

Essa seção instala e carrega os pacotes necessários para a execução do código.

In [ ]:
!pip install pydotplus
!pip install dtreeviz

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn import tree


from google.colab import files
uploaded = files.upload()



Saving State_of_data_BR_2023_sem_cod.xlsx to State_of_data_BR_2023_sem_cod (1).xlsx


### Transformação de dados

Inicialmente os atributos são convertidos para o formato de dicionário.
Em seguida os atributos são transformados em binário, para cada par (key, value) do dicionário.

<code>DictVectorizer(dtype=class ‘numpy.float64’, separator=’=’, sparse=True, sort=True)</code>

A classe é convertida para labels únicos sequenciais.

<code>
 le = preprocessing.LabelEncoder()
  
 le.fit(dados)
</code>


In [ ]:
state_data = pd.read_excel("State_of_data_BR_2023_sem_cod.xlsx", sheet_name=0)
print("\nDimensões: {0}".format(state_data.shape))
print("\nCampos: {0}".format(state_data.keys()))
print(state_data.describe(), sep='\n')

X_dict = state_data.iloc[:,0:4].T.to_dict().values()
vect = DictVectorizer(sparse=False)
X_train = vect.fit_transform(X_dict)

le = LabelEncoder()
y_train = le.fit_transform(state_data.iloc[:,4])

# Exibe o dado convertido em dicionario.
print("Atributos:", X_dict)

# Exibe a estrutura do dado convertido em binário.
print("Shape do dado de treinamento: {0}".format(X_train.shape))

print("Labels:", y_train)




Dimensões: (5037, 38)

Campos: Index(['('P0', 'id')', '('P1_a ', 'Idade')', '('P1_b ', 'Genero')',
       '('P1_c ', 'Cor/raca/etnia')',
       '('P1_e ', 'experiencia_profissional_prejudicada')',
       '('P1_e_3 ', 'Experiencia prejudicada devido a minha identidade de gênero')',
       '('P1_f ', 'aspectos_prejudicados')',
       '('P1_f_1', 'Quantidade de oportunidades de emprego/vagas recebidas')',
       '('P1_f_2', 'Senioridade das vagas recebidas em relação à sua experiência')',
       '('P1_f_3', 'Aprovação em processos seletivos/entrevistas')',
       '('P1_f_4', 'Oportunidades de progressão de carreira')',
       '('P1_f_5', 'Velocidade de progressão de carreira')',
       '('P1_f_6', 'Nível de cobrança no trabalho/Stress no trabalho')',
       '('P1_f_7', 'Atenção dada diante das minhas opiniões e ideias')',
       '('P1_f_8', 'Relação com outros membros da empresa, em momentos de trabalho')',
       '('P1_f_9', 'Relação com outros membros da empresa, em momentos de integra

### Indução do Modelo


Os três passos para indução de um modelo são:

1.   Instanciar o modelo: ``` DecisionTreeClassifier()```
2.   Treinar o modelo: ```fit()```
3.   Testar o modelo: ```predict()```



In [ ]:
treeData = DecisionTreeClassifier(random_state=0, criterion='entropy')
treeData = treeData.fit(X_train, y_train)
print("Acurácia:", treeData.score(X_train, y_train))

y_pred = treeData.predict(X_train)
print("Acurácia de previsão:", accuracy_score(y_train, y_pred))
print(classification_report(y_train, y_pred))

cnf_matrix = confusion_matrix(y_train, y_pred)
labels_reais = le.classes_
cnf_table = pd.DataFrame(
    data=cnf_matrix,
    index=[f"Real: {label}" for label in labels_reais],
    columns=[f"Previsto: {label}" for label in labels_reais]
)

print(cnf_table)


Acurácia: 1.0
Acurácia de previsão: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1417
           1       1.00      1.00      1.00      2434
           2       1.00      1.00      1.00      1186

    accuracy                           1.00      5037
   macro avg       1.00      1.00      1.00      5037
weighted avg       1.00      1.00      1.00      5037

                      Previsto: Não  Previsto: Não respondido  Previsto: Sim
Real: Não                      1417                         0              0
Real: Não respondido              0                      2434              0
Real: Sim                         0                         0           1186


### Exibição da árvore de decisão



In [15]:
from dtreeviz import dtreeviz

viz = dtreeviz(treeData, # Changed 'model' to 'treeData'
              X_train,
              y_train,
              target_name="experiência_profissional_prejudicada",
              get_feature_names=vect.feature_names_out(),
              class_names=list(le.classes_),
              fancy=True)

viz.view()

AttributeError: 'DictVectorizer' object has no attribute 'feature_names_out'

In [ ]:
import pydotplus
# Create DOT data
dot_data = tree.export_graphviz(treeData, out_file=None,
                                feature_names=vect.feature_names_,
                                rounded =True,
                                filled=True,
                                class_names=["sim", "nao"])

# Draw graph
graph = pydotplus.graph_from_dot_data(dot_data)

# Show graph
Image(graph.create_png())
